In [5]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from src.utils import Preprocess
from src.config import MAX_LEN, CHANNELS, NUM_CLASSES, SEQ_LEN
import tensorflow as tf

from src.backbone import TFLiteModel, get_model


In [6]:
NPY_PATH = 'npy_data/'

In [7]:
actions = ['السلام عليكم','اهلا بك','اسف','الحمد لله','اين تسكن','بخير']
label_map = {label:num for num, label in enumerate(actions)}

In [8]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(NPY_PATH, action))).astype(int):
        window = []
        for frame_num in range(SEQ_LEN):
            try:
                res = np.load(os.path.join(NPY_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            except:
                res = window[-1]
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [21]:
import numpy as np

def one_hot_encode_with_padding(categories, num_classes=6, target_dim=250):
    # One-hot encode the categories
    one_hot_encoded = np.eye(num_classes)[categories]
    
    # Create the target array with zeros
    padded_encoded = np.zeros((len(categories), target_dim))
    
    # Assign the one-hot encoded vectors to the beginning of the padded array
    padded_encoded[:, :num_classes] = one_hot_encoded
    
    return padded_encoded

In [22]:
X = np.array(sequences)
y = one_hot_encode_with_padding(labels)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=47, stratify=y)

In [24]:
X_train.shape

(382, 30, 543, 3)

In [25]:
y_train.shape

(382, 250)

In [26]:
x_train_preprocessed = Preprocess()(tf.cast(X_train, dtype=tf.float32))
x_test_preprocessed = Preprocess()(tf.cast(X_test, dtype=tf.float32))

# BUILD THE MODEL

In [30]:
from src.backbone import get_model
model = get_model(max_len=30, output_dim=250)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 30, 708)]    0           []                               
                                                                                                  
 stem_conv (Dense)              (None, 30, 192)      135936      ['input_3[0][0]']                
                                                                                                  
 stem_bn (BatchNormalization)   (None, 30, 192)      768         ['stem_conv[0][0]']              
                                                                                                  
 13_expand_conv (Dense)         (None, 30, 384)      74112       ['stem_bn[0][0]']                
                                                                                            

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [32]:
model.load_weights('models\islr-fp16-192-8-seed42-fold0-best.h5')

In [19]:
x_test_preprocessed.shape

TensorShape([43, 30, 708])

In [35]:
len(model.layers)

67

In [65]:
for layer in model.layers[-1:]:  # Adjust the range based on which layers you want to freeze
    layer.trainable = True

In [66]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 30, 708)]    0           []                               
                                                                                                  
 stem_conv (Dense)              (None, 30, 192)      135936      ['input_3[0][0]']                
                                                                                                  
 stem_bn (BatchNormalization)   (None, 30, 192)      768         ['stem_conv[0][0]']              
                                                                                                  
 13_expand_conv (Dense)         (None, 30, 384)      74112       ['stem_bn[0][0]']                
                                                                                            

In [67]:
# Create an EarlyStopping callback
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_categorical_accuracy',  # Monitor the validation loss
    patience=70,         # Number of epochs with no improvement after which training will be stopped
    mode='max',          # Minimize the monitored metric
    restore_best_weights=True  # Restore the weights of the best epoch
)

# Train the model with early stopping
history = model.fit(
    x_train_preprocessed, y_train,
    epochs=1000,
    batch_size=64,
    validation_data=(x_test_preprocessed, y_test),
    callbacks=[early_stopping]  # Add the callback to the fit method
)

Epoch 1/1000
6/6 [==============================] - 1s 92ms/step - loss: 0.4605 - categorical_accuracy: 0.7670 - val_loss: 0.5236 - val_categorical_accuracy: 0.7442
Epoch 2/1000
6/6 [==============================] - 0s 67ms/step - loss: 0.4518 - categorical_accuracy: 0.7749 - val_loss: 0.4961 - val_categorical_accuracy: 0.7907
Epoch 3/1000
6/6 [==============================] - 0s 60ms/step - loss: 0.4546 - categorical_accuracy: 0.7880 - val_loss: 0.4995 - val_categorical_accuracy: 0.7442
Epoch 4/1000
6/6 [==============================] - 0s 63ms/step - loss: 0.4302 - categorical_accuracy: 0.7827 - val_loss: 0.4845 - val_categorical_accuracy: 0.7442
Epoch 5/1000
6/6 [==============================] - 0s 62ms/step - loss: 0.4424 - categorical_accuracy: 0.7775 - val_loss: 0.4893 - val_categorical_accuracy: 0.7907
Epoch 6/1000
6/6 [==============================] - 0s 66ms/step - loss: 0.4424 - categorical_accuracy: 0.7644 - val_loss: 0.4996 - val_categorical_accuracy: 0.7442
Epoch 7/10

In [68]:
model.evaluate(x_test_preprocessed, y_test)

2/2 [==============================] - 0s 90ms/step - loss: 0.4961 - categorical_accuracy: 0.7907


[0.49608930945396423, 0.7906976938247681]